# Model 

In [ ]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torchvision 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader 
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm 
import cv2
from torchvision.utils import save_image
import os
import pandas as pd
import numpy as np 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d, num_classes, img_size):
        super(Discriminator, self).__init__()
        self.img_size = img_size
        self.disc = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(channels_img+1, features_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
        )
        #this embedding is described as a visual stamp that the model can use to learn, basically, the class of the image
        self.embed = nn.Embedding(num_classes, img_size*img_size) 


    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x, labels):
        embedding = self.embed(labels).view(labels.shape[0], 1, self.img_size, self.img_size) #channel = 1 
        x = torch.cat([x, embedding], dim=1) #dims: N(batch-size) x img_channel(s) x H x W 
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g, num_classes, img_size, embed_size):
        super(Generator, self).__init__()
        self.img_size = img_size
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise+embed_size, features_g * 16, 4, 1, 0),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )
        self.embed = nn.Embedding(num_classes, embed_size) #what is this class? 

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x, labels):
        #latent vector z: N * noise_dim * 1 * 1
        embedding = self.embed(labels).unsqueeze(2).unsqueeze(3) #what does unsqueeze do? 
        x = torch.cat([x, embedding], dim=1)
        return self.net(x)


def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)


def test():
    N, in_channels, H, W = 8, 3, 64, 64
    noise_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
    gen = Generator(noise_dim, in_channels, 8)
    z = torch.randn((N, noise_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"


In [ ]:
def gradient_penalty(critic, labels, real, fake, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = real * alpha + fake * (1 - alpha)

    # Calculate critic scores
    mixed_scores = critic(interpolated_images, labels)

    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty

In [ ]:
# Model class must be defined somewhere
import torchvision.transforms as transforms 

# Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 2e-4
BATCH_SIZE = 64
IMG_SIZE = 64
CHANNELS_IMG = 3
#new for the conditional gan
NUM_CLASSES = 7
GEN_EMBEDDING = 100
# end here 
Z_DIM = 100
NUM_EPOCHS = 55 #test with this num (looks best at 32) maybe 80? 55? 
FEATURES_CRITIC = 16
FEATURES_GEN = 16
CRITIC_ITERATIONS = 5
LAMBDA_GP = 10


# initialize gen and disc, note: discriminator should be called critic,
# according to WGAN paper (since it no longer outputs between [0, 1])
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN, NUM_CLASSES, IMG_SIZE, GEN_EMBEDDING).to(device)
critic = Discriminator(CHANNELS_IMG, FEATURES_CRITIC, NUM_CLASSES, IMG_SIZE).to(device)

#path for gen model 
gen_path = "/content/drive/MyDrive/HAM10000/models/GAN.pt"
gen.load_state_dict(torch.load(gen_path))

# model.eval()


RuntimeError: ignored

In [ ]:
# #create image 
# noise = torch.randn(1, Z_DIM, 1, 1).to(device)
# label = torch.tensor([5]).to(device)
# fake = gen(noise, label)

# #small test here
upscale = transforms.Compose([
    transforms.Resize(64*4),
    ])
# fake = upscale(fake)
# fake = torchvision.utils.make_grid(fake, normalize=True)

# #save image
# img_path = "/content/drive/MyDrive/HAM10000/GAN_Images/image.png"
# save_image(fake, img_path)

In [ ]:
def create_image(label, path):
  #create noise and label
  noise = torch.randn(1, Z_DIM, 1, 1).to(device)
  label = torch.tensor([label]).to(device)
  #produce image
  fake = gen(noise, label)
  #upscale image
  upscale = transforms.Compose([
      transforms.Resize(64*4),
      ])
  fake = upscale(fake)
  #normalize
  fake = torchvision.utils.make_grid(fake, normalize=True)
  #save image
  save_image(fake, str("/content/drive/MyDrive/HAM10000/GAN_Images_2/" + path))

In [ ]:
gan_df = pd.DataFrame()
ids = []
dxs = []

In [ ]:
thresh = 100
# {'bkl': 0, 'nv': 1, 'df': 2, 'mel': 3, 'vasc': 4, 'bcc': 5, 'akiec': 6}
minority_classes = [0, 2, 3, 4, 5, 6]
image_ids = {} #map img_id ->  dx
#starting image = 0 
num = 0 
for c in minority_classes:
  for i in tqdm(range(0, thresh + 1)):
    #make the filename
    id = str(num).zfill(7)
    id = str("GAN_" + id+".png")
    #make and save the image
    create_image(c, id)
    #add the information to the 
    ids.append(id)
    dxs.append(c)
    num += 1

In [ ]:
gan_df["image_id"] = ids
gan_df["dx"] = dxs

In [ ]:
gan_df

In [ ]:
#forgot to save df 
gan_df.to_csv("/content/drive/MyDrive/HAM10000/HAM10000_GAN_data_2.csv")

In [ ]:
import os
os.chdir("/content/drive/MyDrive/HAM10000/GAN_Images_2/")
!ls -1 | wc -l